In [2]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import itertools
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [9]:
FIGURE_PATH = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Code/figures/"
DATA_PATH = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/"
DATA_LOAD_PATH = DATA_PATH + "original_data/"
INPUT_MODEL_PATH = DATA_PATH + "processed_data/"
MODEL_SAVE_PATH = DATA_PATH + "trained_models/"
RESULTS_PATH = DATA_PATH + "model_results/"
path_to_shapefile = "C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/ne_110m_admin_0_countries.shp"
world = gpd.read_file(path_to_shapefile)
with open("functions_training_model.py", 'r') as file:
    content = file.read()

# Execute the content of the .py file
exec(content)
df_empirical_Holocene = pd.read_csv(INPUT_MODEL_PATH + "df_empirical_Holocene.csv")
df_global_grid = pd.read_csv(INPUT_MODEL_PATH + "df_global_grid.csv")
df_empirical_holocene= gpd.read_file("C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/df_empirical_holocene.geojson")
df_validation=gpd.read_file("C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/df_empirical_holocene_val.geojson")
df_wind = pd.read_csv(INPUT_MODEL_PATH + "df_wind.csv", usecols=['wind', 'latitude'])

latitude_wind = torch.tensor(df_wind['latitude'].values / 90, dtype=torch.float32)
mean_wind = torch.tensor(df_wind['wind'].values / df_wind['wind'].max(), dtype=torch.float32)


def wind_func(lat_tensor):
    # Interpolazione lineare in PyTorch
    return torch.interp(lat_tensor.flatten(), latitude_wind, mean_wind).unsqueeze(1)





FileNotFoundError: [Errno 2] No such file or directory: 'functions_training_model.py'

In [5]:
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.activation = nn.SiLU()
        layer_list = []

        for i in range(len(layers) - 1):
            layer_list.append(nn.Linear(layers[i], layers[i + 1]))
            if i < len(layers) - 2:
                layer_list.append(self.activation)
        self.model = nn.Sequential(*layer_list)

    def forward(self, x):
        return self.model(x)


In [6]:
# PDE
D = torch.nn.Parameter(torch.tensor(1.0, requires_grad=True))


def pde_residual(model, x, wind_func, D):
    x.requires_grad_(True)
    u = model(x)

    grad_u = autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    # Derivate parziali
    du_dx = grad_u[:, 0:1]
    du_dy = grad_u[:, 1:2]

    # Secondi ordini
    d2u_dx2 = autograd.grad(du_dx, x, grad_outputs=torch.ones_like(du_dx), create_graph=True)[0][:, 0:1]
    d2u_dy2 = autograd.grad(du_dy, x, grad_outputs=torch.ones_like(du_dy), create_graph=True)[0][:, 1:2]

    # Coefficiente K in funzione della latitudine (seconda colonna di x)
    K = wind_func(x[:, 1:2])

    # Residuo PDE (esempio diffusione anisotropa)
    residual = (-K * du_dx * (1 / torch.cos(x[:, 1:2] * np.pi / 2)) + D * (
        (1 / (torch.cos(x[:, 1:2] * np.pi / 2) ** 2) * d2u_dx2 + d2u_dy2 - torch.tan(x[:, 1:2] * np.pi / 2) * du_dy)))
    return residual


In [7]:
# === PARAMETRI DEL DOMINIO ===
x_min, x_max = -2.0, 2.0
y_min, y_max = -0.89, 0.89

# === 1. DATI OSSERVATI ===
# Assumi che lon/lat siano già scalati correttamente in [-2, 2] e [-0.89, 0.89]
X_obs = torch.tensor(df_empirical_Holocene[['lon', 'lat']].values / 90, dtype=torch.float32)
Y_obs = torch.tensor(df_empirical_Holocene['log_dep_norm'].values.reshape(-1, 1), dtype=torch.float32)

# Verifica range
print("Range X_obs:", X_obs[:, 0].min().item(), X_obs[:, 0].max().item())
print("Range Y_obs:", X_obs[:, 1].min().item(), X_obs[:, 1].max().item())

# === 2. PUNTI INTERNI (PDE) ===
N_pde = 3000

xy_pde = torch.cat([
    torch.rand(N_pde, 1) * (x_max - x_min) + x_min,  # x in [x_min, x_max]
    torch.rand(N_pde, 1) * (y_max - y_min) + y_min  # y in [y_min, y_max]
], dim=1)

# === 3. CONDIZIONI AL CONTORNO DIRICHLET ===

# Nord: y = y_max
N_bc = 200
X_north = torch.rand(N_bc, 1) * (x_max - x_min) + x_min
Y_north = torch.full_like(X_north, y_max)
X_bc_north = torch.cat([X_north, Y_north], dim=1)
Y_bc_north = torch.full((N_bc, 1), -1.0)  # Valore BC nord

# Sud: y = y_min
X_south = torch.rand(N_bc, 1) * (x_max - x_min) + x_min
Y_south = torch.full_like(X_south, y_min)
X_bc_south = torch.cat([X_south, Y_south], dim=1)
Y_bc_south = torch.full((N_bc, 1), -2.0)  # Valore BC sud

# === 4. CONDIZIONI PERIODICHE (x = x_min e x = x_max) ===
N_periodic = 200
Y_periodic = torch.linspace(y_min, y_max, N_periodic).unsqueeze(1)

X_periodic_left = torch.full_like(Y_periodic, x_min)
X_periodic_right = torch.full_like(Y_periodic, x_max)

X_periodic_left = torch.cat([X_periodic_left, Y_periodic], dim=1)
X_periodic_right = torch.cat([X_periodic_right, Y_periodic], dim=1)


NameError: name 'df_empirical_Holocene' is not defined

In [ ]:
# K in funzione della latitudine (normalizzata)
def wind_func(lat_tensor):
    return 1.0 + 0.5 * torch.sin(np.pi * lat_tensor)  # funzione fittizia


In [ ]:
def total_loss(model, X_obs, Y_obs, xy_pde, X_bc1, Y_bc1,
               X_bc2, Y_bc2, X_periodic_left, X_periodic_right,
               wind_func, D,
               lambda_obs=5.0, lambda_pde=10.0, lambda_bc1=0.5, lambda_bc2=0.5,
               lambda_per0=1.0, lambda_per1=1.0):
    # Loss osservati
    pred_obs = model(X_obs)
    loss_obs = nn.MSELoss()(pred_obs, Y_obs)

    # PDE residual
    res_pde = pde_residual(model, xy_pde, wind_func, D)
    loss_pde = torch.mean(res_pde ** 2)

    # BC1 Dirichlet (esempio)
    pred_bc1 = model(X_bc1)
    loss_bc1 = nn.MSELoss()(pred_bc1, Y_bc1)

    # BC2 Dirichlet (esempio)
    pred_bc2 = model(X_bc2)
    loss_bc2 = nn.MSELoss()(pred_bc2, Y_bc2)

    # Condizione periodica sul valore (ordine derivata 0)
    u_left = model(X_periodic_left)
    u_right = model(X_periodic_right)
    loss_per0 = nn.MSELoss()(u_left, u_right)

    # Condizione periodica sulla derivata (ordine derivata 1)
    # Calcolo derivata in X_periodic_left e X_periodic_right
    x_left = X_periodic_left.clone().detach().requires_grad_(True)
    x_right = X_periodic_right.clone().detach().requires_grad_(True)

    u_left = model(x_left)
    u_right = model(x_right)

    grad_left = autograd.grad(u_left, x_left, grad_outputs=torch.ones_like(u_left), create_graph=True)[0][:, 0:1]
    grad_right = autograd.grad(u_right, x_right, grad_outputs=torch.ones_like(u_right), create_graph=True)[0][:, 0:1]

    loss_per1 = nn.MSELoss()(grad_left, grad_right)

    # Somma pesata
    total = (lambda_obs * loss_obs + lambda_pde * loss_pde +
             lambda_bc1 * loss_bc1 + lambda_bc2 * loss_bc2 +
             lambda_per0 * loss_per0 + lambda_per1 * loss_per1)

    return total, loss_obs.item(), loss_pde.item(), loss_bc1.item(), loss_bc2.item(), loss_per0.item(), loss_per1.item()


In [ ]:
# creo una griglia 1°x1°
# Definisci i range
lon_vals = np.arange(-180, 180, 1)  # Include 180
lat_vals = np.arange(-90, 90, 1)  # Include 90

# Crea la griglia con tutti i punti (lon, lat)
lon_grid, lat_grid = np.meshgrid(lon_vals, lat_vals)
lon_flat = lon_grid.flatten()
lat_flat = lat_grid.flatten()

# Crea il DataFrame
df_global_grid_1x1 = pd.DataFrame({
    'lon': lon_flat,
    'lat': lat_flat
})

print(f"Griglia creata con {len(df_global_grid_1x1)} punti")

# Opzionale: salva su CSV
#df_global_grid_1x1.to_csv("global_grid_1x1.csv", index=False)


In [ ]:
df_empirical_holocene= gpd.read_file("C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/df_empirical_holocene.geojson")
df_validation=gpd.read_file("C:/Users/vitas/Desktop/LE PINN/pinn.global.dust/pinn.global.dust/Data/df_empirical_holocene_val.geojson")
df_training=df[~df['id'].isin(df_validation['id'])]

In [ ]:
# 4. Allena modello con i  pesi e salva la storia delle loss
model = PINN(layers=[2, 32, 32, 32, 32, 32, 1])
D = torch.nn.Parameter(torch.tensor(1.0, requires_grad=True))
params = list(model.parameters()) + [D]
optimizer_adam = torch.optim.Adam(params, lr=1e-4)
#scheduler: scende da 1e-3 fino a 1e-5 in 10000 epoche
scheduler = ReduceLROnPlateau(
    optimizer_adam, mode='min', factor=0.5, patience=500,
    min_lr=1e-7, )

# 3. Definisci i pesi (lambda) manualmente
lambda_obs = 20
lambda_pde = 8
lambda_bc1 = 3
lambda_bc2 = 3
lambda_per0 = 1
lambda_per1 = 1

epochs_adam = 150
loss_history = []

for epoch in range(epochs_adam):
    optimizer_adam.zero_grad()
    loss, l_obs, l_pde, l_bc1, l_bc2, l_per0, l_per1 = total_loss(
        model, X_obs_train, Y_obs_train, xy_pde, X_bc_north, Y_bc_north, X_bc_south, Y_bc_south,
        X_periodic_left, X_periodic_right, wind_func, D,
        lambda_obs=lambda_obs, lambda_pde=lambda_pde, lambda_bc1=lambda_bc1, lambda_bc2=lambda_bc2,
        lambda_per0=lambda_per0, lambda_per1=lambda_per1
    )
    loss.backward()
    optimizer_adam.step()
    scheduler.step(loss.item())
    with torch.no_grad():
        pred_val_epoch = model(X_obs_val)
        val_loss_epoch = nn.MSELoss()(pred_val_epoch, Y_obs_val).item()
    loss_history.append([loss.item(),
                         lambda_obs * l_obs,
                         lambda_pde * l_pde,
                         lambda_bc1 * l_bc1 + lambda_bc2 * l_bc2,
                         lambda_per0 * l_per0,
                         lambda_per1 * l_per1,
                         val_loss_epoch
                         ])

    if epoch % 500 == 0:
        current_lr = optimizer_adam.param_groups[0]['lr']
        print(
            f"Adam | Epoch {epoch:5d} | Total: {loss.item():.5f} | Obs: {l_obs:.5f} | PDE: {l_pde:.5f} | " f"BC: {(l_bc1 + l_bc2):.5f} | Per0: {l_per0:.5f} | Per1: {l_per1:.5f} | "
            f"Val: {val_loss_epoch:.5f}")

